# CO2 Especification Detector

- vamos a crear un modelo de IA para determinar el nivel de pureza de CO2 en el ambito alimenticio, dado por un analisis -ya sea de forma manual o por pdf.- y dar un aviso si pasa de las especificaciones del cliente.

- Datos a tener en cuenta:
    - Para esta primera versión voy a enfocarme en el CO2 liquido.

- Pureza de CO2:
    - 99,9 %

- Niveles de Impurezas permitidos:
    - Humedad (H2O) <= 50 ppm v/v
    - Oxigeno (O2) <= 30 ppm v/v
    - Monoxido de Carbono (CO) <= 10 ppm v/v
    - Óxidos de Nitrogeno (NOx) <= 10 ppm v/v
    - Compuestos Organicos no Volatiles (COV) <= 5ppm v/v
    - Hidrocarburos (CnHm) <= 30 ppm v/v
    - Azufre Total (SO) <= 0.05 ppm v/v

--------

- NOTA:
    - ppm v/v : Partes por millon en volumen.
    - ppm w/w : Partes por millon en peso.

- Link publico de AirLiquide con las especificaciones que utilizan:
    - https://es.airliquide.com/gases/gases-de-calidad-alimentaria-aligal/aligaltm-2-dioxido-de-carbono-co2-de-calidad-alimentaria

- Fecha del Notebook:
    - 26/02/25

# Creación de Datos Sinteticos:

- Al no encontrar datos publicos para esta primera versión vamos a generar datos sinteticos para entrenar al modelo. Es importante tenerlo en cuenta aunque si se entrenan correctamente con las especificaciones correctas debería de dar un modelo funcional.

In [1]:

# Librerías:

import numpy as np
import pandas as pd

In [ ]:
# Generador de Datos Sinteticos

def DataGeneration(max_value, crit_value, seed, min_value= 0, p_outranged = 0.2, n = 1000):

    #Para poder replicar los datos, vamos a establecer una semilla.
    random_seed = np.random.seed(seed)
    
    #Determinamos los datos que estaran o no dentro de las especifiaciones.
    ranged = int (n - p_outranged)
    outranged = int (n - ranged)

    #Generamos los datos. 
    data_ranged = np.random.uniform(min_value, max_value, ranged)
    data_outranged = np.random.uniform(min_value, crit_value, outranged)

    #Juntamos y mezclamos
    data = np.concatenate([data_ranged, data_outranged])
    np.random.shuffle(data)

    return data

- Explicacion de los args:

    - min_value: Valor minimo permitido en las especificaciones. Por defecto 0.
    - max_value: Valor máximo permitido en las especificaciones. Esto hay que ponerlo según el producto.
    - crit_value: Valor superior al permitido en las especificaciones para crear un dataset con datos erroreos, es importante para que el modelo pueda aprender.
    - p_outranged: % de datos fuera del rango. Por defecto el 20%
    - n: numero de muestras, por defecto 1000.

In [ ]:
#Creamos los datos

co2_purify = DataGeneration(min_value = 99.9, max_value = 100, crit_value = 99.8, seed=42)

co2_purify

array([99.9802687 , 99.96513907, 99.93971269, 99.93012265, 99.94156579,
       99.95373827, 99.984213  , 99.99853942, 99.98923555, 99.91108472,
       99.97583871, 99.98622026, 99.95522697, 99.91583953, 99.95721417,
       99.92160139, 99.95219347, 99.90882716, 99.94607962, 99.98026708,
       99.94022243, 99.96316681, 99.90615857, 99.92925841, 99.90061143,
       99.90507512, 99.90190144, 99.9891188 , 99.98608981, 99.99641573,
       99.91117625, 99.91420686, 99.90497686, 99.90264664, 99.944421  ,
       99.96545324, 99.90100105, 99.954551  , 99.94707902, 99.98651442,
       99.93578239, 99.97413288, 99.98638854, 99.93050991, 99.98425727,
       99.94006327, 99.96937322, 99.96804978, 99.95057196, 99.91959038,
       99.92482106, 99.97481446, 99.90325278, 99.96966865, 99.96995152,
       99.98769566, 99.94793575, 99.98724622, 99.9600733 , 99.99050144,
       99.99130565, 99.9997911 , 99.9672766 , 99.96276642, 99.94098608,
       99.93147645, 99.98406157, 99.99203365, 99.91516271, 99.94